In [ ]:
from sys import path
path.append("..")
from itertools import product
from Models.TabularClassifier import TabularClassifier
from Models.Classifier import train,accuracy
from Datasets_DataLoaders.customDataLoaders import dataLoaderTabular
from Utilities.dataformater import DataFormater

config = {
    "l1":[2**i for i in range(3,9)],
    "l2":[2**i for i in range(3,9)],
    "l3":[2**i for i in range(3,9)],
    "l4":[2**i for i in range(3,9)]
}

In [ ]:
def tune(config,trainLoader,valLoader,testLoader,testEpochs=10,learningRate=0.001):
    '''Tunes NN grid search over layer size.
    Will soon implement early stopping, learning rate, batch size, and architecture depth.
    '''
    modelConfigsToTest = list(product(*[config[param] for param in config]))
    bestModel = TabularClassifier()
    bestParams = None
    _,_,bestModelLoss = train(bestModel,trainLoader,valLoader,epochs=testEpochs)
    for modelConfig in modelConfigsToTest:
        model = TabularClassifier(*modelConfig)
        model,trainLoss,valLoss = train(model,trainLoader,valLoader,epochs=testEpochs,lr=learningRate)
        if valLoss<=bestModelLoss:
            bestModel = model
            bestModelLoss = valLoss
            bestParams = modelConfig
    return bestModel,accuracy(bestModel,valLoader),bestParams

X_train,X_validation,X_test,y_train,y_validation,y_test = DataFormater().preProcessing(toNumpy=True)
trainLoader,valLoader,testLoader = dataLoaderTabular(X_train,X_validation,X_test,y_train,y_validation,y_test)

In [ ]:
model,performance,bestParams = tune(config,trainLoader,valLoader,testLoader)
print(f"{performance}:{bestParams}")

In [ ]:
#test model with best architecture
model = TabularClassifier(256,64,128,32)
model,_,_ = train(model,trainLoader,valLoader,epochs=10000,lr=0.01)
accuracy(model,valLoader)